## Generate HScript function completions

Instead of generating it from scratch, I have a complete list
of expression functions. It's easy to keep it up to date by
diffing with list printed by Texport. HScript is barely changing.

In [1]:
import re
import json
import random
import subprocess
from pprint import pprint

In [2]:
with open('expressions.cmd') as f:
    expressions_content = f.read() 

# Check the difference between functions listed in the file
# and functions available in current Houdini exhelp command.
proc = subprocess.run('hbatch -q -c exhelp'.split(),
                      stdout=subprocess.PIPE,
                      universal_newlines=True)
exhelp = set(proc.stdout.split())
listed = set(re.findall(r'^\w+ (\w+)\b.*$', expressions_content, flags=re.M))
print(exhelp - listed)
print(listed - exhelp)

set()
{'iprquerys', 'iprquery'}


In [3]:
expressions = {}

for e in expressions_content.split('\n'):
    match = re.match(r'(\w+) (\w+)\((.*)\)', e)
    r, f, args = match.group(1, 2, 3)
    expressions[f] = {'return': r}
    if args:
        args, argnames = zip(*[pair.split() for pair in args.split(',')])
        expressions[f]['args'], expressions[f]['argnames'] = args, argnames

def test():
    print(len(expressions))
    f = random.choice(list(expressions.keys()))
    print(f)
    pprint(expressions[f])
    
test()

404
opname
{'argnames': ('name',), 'args': ('string',), 'return': 'string'}


In [4]:
comps = {
    'scope': 'source.hscript -string',
    'completions': []
}
unique_triggers = set()

for e in expressions:
    args = expressions[e].get('argnames', [])
    
    trigger = f"{e}({', '.join(args)})"

    # Completion exists.
    if trigger in unique_triggers:
        continue
        
    # Completion not exists. Make a new one.
    unique_triggers.add(trigger)

    # Build contents.
    cargs = [f'${{{i}:{arg}}}' for i, arg in enumerate(args, 1)]
    contents = f"{e}({', '.join(cargs)})"

    comps['completions'].append({'trigger': trigger,
                                 'contents': contents})


print('Generated %d completions.' % len(unique_triggers))


# Write completions into a functions.sublime-completions file.
with open('expressions.sublime-completions', 'w') as f:
    json.dump(comps, f, indent=4)

Generated 404 completions.
